In [1]:
import os
import cv2
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.models import Sequential
from keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [2]:
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  # crawls inside folders
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count
train_dir ="/content/drive/MyDrive/Plant-Leaf-Disease-Prediction-main/tomato/train"
test_dir="/content/drive/MyDrive/Plant-Leaf-Disease-Prediction-main/tomato/test"

In [3]:
 #train file image count
train_samples =get_files(train_dir)
#to get tags
num_classes=len(glob.glob(train_dir+"/*"))
#test file image count
test_samples=get_files(test_dir)
print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")

10 Classes
10000 Train images
100 Test images


In [4]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )
test_datagen=ImageDataGenerator(rescale=1./255)

In [6]:
input_shape=(1,128,128,3)
train_generator =train_datagen.flow_from_directory(train_dir,target_size=(128,128),batch_size=32)
test_generator=test_datagen.flow_from_directory(test_dir,shuffle=True,target_size=(128,128),batch_size=32)

Found 10000 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


In [7]:
IMAGE_SIZE = [128,128]
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# We don't need to train existing weights
for layer in inception.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 3s 0us/step


In [8]:

# Model layers -> can add more if required
x = Flatten()(inception.output)
prediction = Dense(num_classes, activation='softmax')(x)
# Create a model object
model = Model(inputs=inception.input, outputs=prediction)
# View the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 63, 63, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 63, 63, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 63, 63, 32)           0         ['batch_normalization[0][0

In [9]:

validation_generator = train_datagen.flow_from_directory(
                       test_dir,
                       target_size=(128,128),
                       batch_size=32)

Found 100 images belonging to 10 classes.


In [10]:
# Defining the cost and model optimization method to use
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history4 = model.fit(
    train_generator,#egitim verileri
    steps_per_epoch=None,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=None,
    verbose=1,
    callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.3,patience=3, min_lr=0.000001)],
    shuffle=True)


Epoch 1/5
313/313 [==============================] - 5328s 17s/step - loss: 1.7780 - accuracy: 0.5979 - val_loss: 2.5235 - val_accuracy: 0.5700 - lr: 0.0010
Epoch 2/5
313/313 [==============================] - 365s 1s/step - loss: 1.3499 - accuracy: 0.7001 - val_loss: 1.1973 - val_accuracy: 0.7100 - lr: 0.0010
Epoch 3/5
313/313 [==============================] - 363s 1s/step - loss: 1.3706 - accuracy: 0.7197 - val_loss: 1.9310 - val_accuracy: 0.6300 - lr: 0.0010
Epoch 4/5
313/313 [==============================] - 363s 1s/step - loss: 1.2163 - accuracy: 0.7402 - val_loss: 1.4409 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 5/5
313/313 [==============================] - 364s 1s/step - loss: 1.2151 - accuracy: 0.7536 - val_loss: 1.6692 - val_accuracy: 0.6900 - lr: 0.0010


In [11]:
model.save('/content/drive/MyDrive/Plant-Leaf-Disease-Prediction-main/models/Inceptionv3.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:

print("[INFO] Calculating model accuracy")
scores = model.evaluate(validation_generator)
print(f"Accuracy: {scores[1]*100}")

[INFO] Calculating model accuracy
4/4 [==============================] - 4s 641ms/step - loss: 1.5001 - accuracy: 0.7300
Accuracy: 73.00000190734863
